In [26]:
using CSV
import Pkg
Pkg.add("JSON3")
using JSON3
using DataFrames


# Read the Locations CSV file
df = CSV.read("Locations_test.csv", DataFrame)

# Function to parse JSON strings into Julia objects
function parse_json_string(str)
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    return JSON3.read(str)
end

# Process each location's data
locations_data = Dict()

for row in eachrow(df)
    location_id = row.Location_ID
    
    # Process workers data
    workers = parse_json_string(row.Workers)
    workers_dict = Dict()
    for worker in workers
        workers_dict[worker["Worker_ID"]] = Dict(
            "Capabilities" => worker["Capabilities"]
        )
    end
    
    # Process machines data
    machines = parse_json_string(row.Machines)
    machines_dict = Dict()
    for machine in machines
        machines_dict[machine["Machine_ID"]] = Dict(
            "Capabilities" => machine["Capabilities"],
            "Breakdown_Prob" => machine["Breakdown_Prob"]
        )
    end
    
    # Process inventory resources
    inventory = parse_json_string(row.Inventory_Resources)
    
    # Store all data for this location
    locations_data[location_id] = Dict(
        "workers" => workers_dict,
        "machines" => machines_dict,
        "inventory" => inventory
    )
end

# Create helper functions
function get_all_location_ids()
    return collect(keys(locations_data))
end

function get_all_workers_ids(location_id)
    return collect(keys(locations_data[location_id]["workers"]))
end

function get_worker_capabilities(location_id, worker_id)
    return locations_data[location_id]["workers"][worker_id]["Capabilities"]
end

function get_all_machines_ids(location_id)
    return collect(keys(locations_data[location_id]["machines"]))
end

function get_machine_capabilities(location_id, machine_id)
    return locations_data[location_id]["machines"][machine_id]["Capabilities"]
end

function get_all_resources_ids(location_id)
    return sort(collect(keys(locations_data[location_id]["inventory"])))
end

function get_resource_amounts(location_id, resource_id)
    return locations_data[location_id]["inventory"][resource_id]
end

for l in get_all_location_ids()
    println(get_all_resources_ids(l))
end


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


[:Paint, :Plastic, :Steel]


In [ ]:
using CSV
using JSON3
using DataFrames

# Read the CSV file
df = CSV.read("Projects_test.csv", DataFrame)

# Function to clean and parse JSON strings into Julia objects
function parse_json_string(str)
    # Clean the string: remove newlines and extra spaces
    str = replace(str, r"\s+" => " ")  # Replace multiple spaces with single space
    str = replace(str, "\n" => "")     # Remove newlines
    str = strip(str)                   # Remove leading/trailing whitespace
    
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    
    try
        return JSON3.read(str)
    catch e
        println("Error parsing JSON string: ", str)
        rethrow(e)
    end
end

# Process each project's data
projects_data = Dict()

for row in eachrow(df)
    project_id = strip(row.Project_ID)
    
    # Process all JSON fields
    total_resources = parse_json_string(row.Total_Resources)
    workers_req = parse_json_string(row.Workers_Requirement)
    machines_req = parse_json_string(row.Machines_Requirement)
    stages = parse_json_string(row.Stages)

    # Create a dictionary of stages indexed by Stage_ID
    stages_dict = Dict()
    for stage in stages
        stage_id = stage["Stage_ID"]
        stages_dict[stage_id] = stage
    end
    
    # Store all data for this project
    projects_data[project_id] = Dict(
        "total_resources" => total_resources,
        "workers_requirement" => workers_req,
        "machines_requirement" => machines_req,
        "deadline" => row.Deadline,
        "customer_priority" => row.Customer_Priority,
        "stages" => stages_dict,
        "revenue" => row.Revenue
    )
end


function get_all_project_ids()
    return sort(collect(keys(projects_data)))
end

# Create helper functions
function get_project_data(project_id)
    return projects_data[project_id]
end

function get_project_resources(project_id)
    return projects_data[project_id]["total_resources"]
end

function get_project_stages_ids(project_id)
    return sort(collect(keys(projects_data[project_id]["stages"])))
end

function get_project_stages_length(project_id)
    return length(get_project_stages_ids(project_id))
end

function get_project_stage_workers(project_id, stage_id)
    stages = projects_data[project_id]["stages"][stage_id]["Workers_Needed"]
end

####Use this one when indexing through stages
function get_project_stage_workers_length(project_id, stage_id)
    return length(get_project_stage_workers(project_id, stage_id))
end

function get_project_stage_machines(project_id, stage_id)
    stages = projects_data[project_id]["stages"][stage_id]["Machines_Needed"]
end

function get_project_stage_machines_length(project_id, stage_id)
    return length(get_project_stage_machines(project_id, stage_id))
end

function get_project_stage_duration(project_id, stage_id)
    stages = projects_data[project_id]["stages"][stage_id]["Duration_days"]
end

function get_project_deadline(project_id)
    return projects_data[project_id]["deadline"]
end

function get_project_priority(project_id)
    return projects_data[project_id]["customer_priority"]
end

function get_project_revenue(project_id)
    return projects_data[project_id]["revenue"]
end

function get_project_resources_ids(project_id)
    return sort(collect(keys(projects_data[project_id]["total_resources"])))
end

function get_project_resources_amount(project_id, resource_id)
    return projects_data[project_id]["total_resources"][resource_id]
end

function get_project_priority(project_id)
    return projects_data[project_id]["customer_priority"]
end




1
3
2
2


In [28]:
#Preparing the rest of the Parameters

import Pkg
using JuMP, DataFrames, CSV

########################################################
# Important Parameter, this is the end of the job scheduler
deadline = 60
T = 0:deadline
########################################################

# Get all locations
locations = collect(keys(locations_data))

# Create location-specific sets for workers and machines
location_workers = Dict(
    loc => get_all_workers_ids(loc)
    for loc in locations
)

location_machines = Dict(
    loc => get_all_machines_ids(loc)
    for loc in locations
)

# Get all projects and their stages
projects = get_all_project_ids()
project_stages = Dict(
    p => get_project_stages_ids(p) 
    for p in projects
)


Dict{String3, Vector{Any}} with 4 entries:
  "P2" => [1, 2, 3]
  "P1" => [1, 2]
  "P4" => [1, 2, 3, 4]
  "P3" => [1, 2]



## Sets and Indices
- `p ∈ projects`: Set of all projects
- `s ∈ S[p]`: Set of stages for project p
- `l ∈ locations`: Set of all locations
- `m ∈ location_machines[l]`: Set of machines at location l
- `wr ∈ location_workers[l]`: Set of workers at location l
- `t ∈ T`: Set of time periods (1 to deadline)


## Variables
- `y[p,s,l,m,t]`: Binary variable = 1 if machine m at location l is assigned to stage s of project p at time t
- `z[p,s,l,w,t]`: Binary variable = 1 if worker w at location l is assigned to stage s of project p at time t
- `x[p]`: Binary variable = 1 if project p is accepted
- `w[p,t]`: Binary variable = 1 if project p starts at time t
- `u[p,s]`: Integer variable representing the start time of stage s of project p
- `E[p]`: Earnings of the project p if completed 

## Objective Function
Maximize total revenue from accepted projects:
```math
 \max \sum_{p \in P} \text{E}_p \cdot x_p 
```

## Constraints

### 1. Machine Capacity Constraints
Each machine can only be assigned to one project-stage at a time:
```math
\sum_{p \in P} \sum_{s \in S_p} y_{p,s,l,m,t} \leq 1 \quad \forall  l,m,t
```

### 2. Worker Capacity Constraints
Each worker can only be assigned to one project-stage at a time:
```math
\sum_{p \in P} \sum_{s \in S_p} z_{p,s,l,wr,t} \leq 1 \quad \forall l,w,t
```

### 3. Machine Capability Constraints
Machines can only be assigned to stages requiring their capabilities:
```math
y_{p,s,l,m,t} = 0 \quad \text{if machine } m \text{ doesn't have required capabilities}
```

### 4. Worker Capability Constraints
Workers can only be assigned to stages requiring their capabilities:
```math
z_{p,s,l,wr,t} = 0 \quad \text{if worker } w \text{ doesn't have required capabilities}
```

### 5. Project Start Time Constraints
Each accepted project must start exactly once:
```math
\sum_{t \in T} w_{p,t,-1} \geq x_p \quad \forall p
```

### 6. Stage Start Time Linking Constraints
First stage must start when project starts:
```math
\text{u}_{p,s} = t \cdot w_{p,s,t} \quad \forall p,t,s
```

### 7. Stage Sequence Constraints
Each stage must start after previous stage ends:
```math
\text{u}_{p,s} \geq \text{u}_{p,s-1} + \text{duration}_{p,s-1} \quad \forall p,s>1
```

### 8. Deadline Constraints
Each stage must complete before project deadline:
```math
\text{u}_{p, s_{last}} + \text{duration}_{p, s_{last}} \leq \text{deadline}_p \quad \forall p,s
```

### 9. Labor and Machine Assignment Timing Constraints
Workers and machines must be assigned at stage start times:
```math
\sum_{t \in T} \cdot z_{p,s,l,wr,t} \geq W * w_{p,s,t} \quad \forall p,s,l,wr
```
```math
\sum_{t \in T} \cdot y_{p,s,l,m,t} \geq M * w_{p,s,t} \quad \forall p,s,l,m
```

### 10. Resource Availability

Need to say all stages must consume less than or equal to the amount of avaliable resources

Below is the working Model!!!
## Mathematical Formulation

### Decision Variables
- $y_{p,s,l,m,t} \in \{0,1\}$: Machine assignment
- $z_{p,s,l,wr,t} \in \{0,1\}$: Worker assignment
- $x_p \in \{0,1\}$: Project acceptance
- $u_{p,s} \in \mathbb{Z}_+$: Stage start time
- $I1_p, I2_{p,s}, I3_{p,s} \in \{0,1\}$: Feasibility indicators
- $is\_stage\_active_{p,s,t} \in \{0,1\}$: Stage activity

### Objective
$$\max \sum_{p \in P} \text{revenue}_p \cdot x_p$$

### Constraints

1. Resource Uniqueness:
$$\sum_{p,s} y_{p,s,l,m,t} \leq 1, \sum_{p,s} z_{p,s,l,wr,t} \leq 1$$

2. Capabilities:
$$y_{p,s,l,m,t} = 0 \text{ if } \text{cap}(m) \cap \text{req}(s) = \emptyset$$
$$z_{p,s,l,wr,t} = 0 \text{ if } \text{cap}(wr) \cap \text{req}(s) = \emptyset$$

3. Stage Sequence:
$$u_{p,s} \geq u_{p,s-1} + \text{dur}_{p,s-1}$$

4. Deadline:
$$u_{p,s_{last}} + \text{dur}_{p,s_{last}} - \text{deadline}_p \leq \text{deadline}_p \cdot (1-I1_p)$$

5. Worker Assignment:
$$\sum_{wr,t} z_{p,s,l,wr,t} - \text{dur}_{p,s} \geq -\text{dur}_{p,s} \cdot (1-I2_{p,s})$$
$$\sum_{wr,t} z_{p,s,l,wr,t} \leq \text{dur}_{p,s}$$

6. Machine Assignment:
$$\sum_{m,t} y_{p,s,l,m,t} - \text{dur}_{p,s} \geq -\text{dur}_{p,s} \cdot (1-I3_{p,s})$$
$$\sum_{m,t} y_{p,s,l,m,t} \leq \text{dur}_{p,s}$$

7. Project Acceptance:
$$I1_p \geq x_p, I2_{p,s} \geq x_p, I3_{p,s} \geq x_p$$

8. Stage Activity:
$$is\_stage\_active_{p,s,t} \leq 1 - \frac{u_{p,s} - t}{\text{deadline}}$$
$$is\_stage\_active_{p,s,t} \leq 1 - \frac{t - (u_{p,s} + \text{dur}_{p,s} - 1)}{\text{deadline}}$$

9. Resource Timing:
$$y_{p,s,l,m,t} \leq is\_stage\_active_{p,s,t}$$
$$z_{p,s,l,wr,t} \leq is\_stage\_active_{p,s,t}$$

10. Resource Capacity:
$$\sum_{m} y_{p,s,l,m,t} \leq 1, \sum_{wr} z_{p,s,l,wr,t} \leq 1$$

In [ ]:
#Preparing the rest of the Parameters
using JuMP, HiGHS

# Create model
model = Model(HiGHS.Optimizer)

# Decision variables - now with location-specific workers and machines
@variable(model, y[p in projects, s in project_stages[p], l in locations, m in location_machines[l], t in T], Bin)  # Machine assignments
@variable(model, z[p in projects, s in project_stages[p], l in locations, wr in location_workers[l], t in T], Bin)  # Worker assignments
@variable(model, x[projects], Bin)  # Project acceptance
@variable(model, 0 <= stage_start[p in projects, s in project_stages[p]] <= get_project_deadline(p), Int)
@variable(model, I1[p in projects], Bin)
@variable(model, I2[p in projects, s in project_stages[p]], Bin)
@variable(model, I3[p in projects, s in project_stages[p]], Bin)

# Objective function
@objective(model, Max, sum(get_project_revenue(p) * x[p] for p in projects))

#Worker and Machine Uniqueness Constraints
@constraint(model, [l in locations, m in location_machines[l], t in T], sum(y[p,s,l,m,t] for p in projects, s in project_stages[p]) <= 1)
@constraint(model, [l in locations, wr in location_workers[l], t in T], sum(z[p,s,l,wr,t] for p in projects, s in project_stages[p]) <= 1)

#Machine and Worker Capability Constraints
for l in locations, p in projects, s in project_stages[p]

    stage_machines_needed = get_project_stage_machines(p,s)
    for m in location_machines[l]
        machine_capabilities = get_machine_capabilities(l,m)
        if isempty(intersect(stage_machines_needed, machine_capabilities)) ####Ensures if the machine has a list of skills, there still exists no identical skill in the stage
            @constraint(model, [t in T], y[p,s,l,m,t] == 0)
        end
    end

    stage_workers_needed = get_project_stage_workers(p, s)
    for wr in location_workers[l]
        worker_capabilities = get_worker_capabilities(l,wr)
        if isempty(intersect(worker_capabilities, stage_workers_needed))
            @constraint(model, [t in T], z[p,s,l,wr,t] == 0)
        end
    end
end

for p in projects
    for s in project_stages[p]
        if s > 1
            @constraint(model, stage_start[p, s] >= stage_start[p, s-1] + get_project_stage_duration(p, s-1))
        end
    end
end

#@constraint(model, [p in projects, s in project_stages[p] if s > 1, t in T], stage_start[p,s] >= stage_start[p,s-1] + get_project_stage_duration(p,s-1))

# =====> D1[p] = 1
#@constraint(model, [p in projects, s in last(project_stages[p]), t in T],  stage_start[p,s] <= get_project_deadline(p) - get_project_stage_duration(p,s))
@constraint(model, [p in projects, s in last(project_stages[p]), t in T],  stage_start[p,s] - (get_project_deadline(p) - get_project_stage_duration(p,s)) <= (get_project_deadline(p) - get_project_stage_duration(p,s))*(1-I1[p]))
#@constraint(model, [p in projects, s in last(project_stages[p]), t in T],  stage_start[p,s] - (get_project_deadline(p) - get_project_stage_duration(p,s)) >= -1*(get_project_deadline(p) - get_project_stage_duration(p,s))*I1[p] + .0001*(1-I1[p]))

# =====> D2[p] = 1
#@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) - get_project_stage_duration(p,s) >= 0)
@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) - get_project_stage_duration(p,s) >= (-get_project_stage_duration(p,s))*(1-I2[p,s]))
@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) <= get_project_stage_duration(p,s))
#@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) - get_project_stage_duration(p,s) <= (deadline*length(location_workers[l]) - get_project_stage_duration(p,s))*I2[p,s] - 0.0001*(1-I2[p,s]))

# =====> D3[p] = 1
#@constraint(model, [l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) >= get_project_stage_duration(p,s))
@constraint(model, [p in projects,l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) - get_project_stage_duration(p,s) >= -1*get_project_stage_duration(p,s)*(1-I3[p,s]))
@constraint(model, [p in projects,l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) <= get_project_stage_duration(p,s))
#@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) - get_project_stage_duration(p,s) <= ((deadline+1)*length(location_machines[l]) - get_project_stage_duration(p,s))*I3[p,s] - 0.0001*(1-I3[p,s]))

#@constraint(model, [p in projects], I1[p] + sum(I2[p,s] for s in project_stages[p]) + sum(I3[p,s] for s in project_stages[p]) <= 2 * length(project_stages[p]) + x[p])

@constraint(model, [p in projects], I1[p] >= x[p])
@constraint(model, [p in projects, s in project_stages[p]], I2[p,s] >= x[p])
@constraint(model, [p in projects, s in project_stages[p]], I3[p,s] >= x[p])

# Add binary variables to represent if a time period is within a stage's duration
@variable(model, is_stage_active[p in projects, s in project_stages[p], t in T], Bin)

# Constraints to set is_stage_active based on stage_start and duration
for p in projects, s in project_stages[p]
    stage_duration = get_project_stage_duration(p,s)
    
    # Set is_stage_active to 1 only during the stage's duration
    for t in T
        # If t < stage_start[p,s], then is_stage_active must be 0
        @constraint(model, is_stage_active[p,s,t] <= 1 - (stage_start[p,s] - t)/deadline)
        
        # If t >= stage_start[p,s] + stage_duration, then is_stage_active must be 0
        @constraint(model, is_stage_active[p,s,t] <= 1 - (t - (stage_start[p,s] + stage_duration - 1))/deadline)
        
        # If stage_start[p,s] = 1, then t < stage_start[p,s] + stage_duration, then is_stage_active can be 1
        #@constraint(model, is_stage_active[p,s,t] >= (t - stage_start[p,s] + 1)/deadline)
        #@constraint(model, is_stage_active[p,s,t] >= (stage_start[p,s] + stage_duration - t)/deadline - 1)
    end

    # Link machine assignments to active time periods
    for l in locations, m in location_machines[l]
        @constraint(model, [t in T], y[p,s,l,m,t] <= is_stage_active[p,s,t])
    end
    
    # Link worker assignments to active time periods
    for l in locations, wr in location_workers[l]
        @constraint(model, [t in T], z[p,s,l,wr,t] <= is_stage_active[p,s,t])
    end
end

@variable(model, machine_assigned[p in projects, s in project_stages[p], l in locations, m in location_machines[l]], Bin)

# Link machine_assigned with y variables
@constraint(model, [p in projects, s in project_stages[p], l in locations, m in location_machines[l]], sum(y[p,s,l,m,t] for t in T) <= get_project_stage_duration(p,s) * machine_assigned[p,s,l,m])

# Ensure only one machine is assigned per stage
@constraint(model, [p in projects, s in project_stages[p], l in locations], sum(machine_assigned[p,s,l,m] for m in location_machines[l]) <= 1)

@variable(model, worker_assigned[p in projects, s in project_stages[p], l in locations, wr in location_workers[l]], Bin)

# Link machine_assigned with y variables
@constraint(model, [p in projects, s in project_stages[p], l in locations, wr in location_workers[l]], sum(z[p,s,l,wr,t] for t in T) <= get_project_stage_duration(p,s) * worker_assigned[p,s,l,wr])

# Ensure only one machine is assigned per stage
@constraint(model, [p in projects, s in project_stages[p], l in locations], sum(worker_assigned[p,s,l,wr] for wr in location_workers[l]) <= 1)

@constraint(model, [l in locations, r in get_all_resources_ids(l)], sum(get_project_resources_amount(p,r) * x[p] for p in projects) <= get_resource_amounts(l,r))

optimize!(model)
for p in projects
    println("Project: ", p, " ", value(x[p]))
end

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-02, 2e+02]
  Cost   [2e+04, 4e+04]
  Bound  [1e+00, 5e+01]
  RHS    [2e-02, 5e+02]
Presolving model
3100 rows, 2180 cols, 12024 nonzeros  0s
2256 rows, 1499 cols, 8411 nonzeros  0s
2068 rows, 1497 cols, 8923 nonzeros  0s
Objective function is integral with scale 0.000666667

Solving MIP model with:
   2068 rows
   1497 cols (1490 binary, 7 integer, 0 implied int., 0 continuous)
   8923 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol             

In [30]:
for p in projects
    for s in project_stages[p]
        for l in locations
            println("Project ", p, " Stage ", s, "      Start Time ", value(stage_start[p,s]), "       Duration ", get_project_stage_duration(p,s), "     Endtime: ", get_project_stage_duration(p,s) + value(stage_start[p,s]), "     workers ", value(sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T)), "       machines ", value(sum(y[p,s,l,m,t] for m in location_machines[l], t in T)))
        end
    end
    println(get_project_deadline(p))
end

#=
for p in projects
    println(value(I1[p]))
    for s in project_stages[p]
        println(value(I2[p,s]))
        println(value(I3[p,s]))
    end
end
=#



Project P1 Stage 1      Start Time -1.7647548183265111e-15       Duration 5     Endtime: 4.999999999999998     workers 5.000000000000049       machines 5.000000000000364
Project P1 Stage 2      Start Time 4.999999999999998       Duration 1     Endtime: 5.999999999999998     workers 1.0000000000000002       machines 1.0
16
Project P2 Stage 1      Start Time -0.0       Duration 5     Endtime: 5.0     workers 5.000000000000003       machines 5.0
Project P2 Stage 2      Start Time 5.0       Duration 4     Endtime: 9.0     workers 4.0       machines 4.0
Project P2 Stage 3      Start Time 9.0       Duration 1     Endtime: 10.0     workers 1.0000000000000004       machines 1.0000000000000004
48
Project P3 Stage 1      Start Time 0.0       Duration 4     Endtime: 4.0     workers 4.0       machines 3.9999999999999987
Project P3 Stage 2      Start Time 20.0       Duration 9     Endtime: 29.0     workers 9.0       machines 9.0
49
Project P4 Stage 1      Start Time 0.0       Duration 9     Endtime

In [31]:
for p in projects
    println("\nProject: ", p)
    for s in project_stages[p]
        println("  Stage: ", s)
        for l in locations
            println("    Location: ", l)
            for m in location_machines[l]
                # Check if this machine is assigned to this project-stage
                if any(value(y[p,s,l,m,t]) > 0.5 for t in T)
                    machine_capabilities = get_machine_capabilities(l,m)
                    println("      Machine ", m, " (Capabilities: ", machine_capabilities, ")")
                    # Print the time periods this machine is working
                    working_times = [t for t in T if value(y[p,s,l,m,t]) > 0.5]
                    println("        Working times: ", working_times)
                end
            end

            for wr in location_workers[l]
                # Check if this machine is assigned to this project-stage
                if any(value(z[p,s,l,wr,t]) > 0.5 for t in T)
                    worker_capabilities = get_worker_capabilities(l,wr)
                    println("      Worker ", wr, " (Capabilities: ", worker_capabilities, ")")
                    # Print the time periods this machine is working
                    working_times = [t for t in T if value(z[p,s,l,wr,t]) > 0.5]
                    println("        Working times: ", working_times)
                end
            end
        end
    end
end


Project: P1
  Stage: 1
    Location: L1
      Machine L1_M2 (Capabilities: ["Painter"])
        Working times: [0, 1, 2, 3, 4]
      Worker L1_W5 (Capabilities: ["Assembler", "Painter", "Packer"])
        Working times: [0, 1, 2, 3, 4]
  Stage: 2
    Location: L1
      Machine L1_M5 (Capabilities: ["Assembler"])
        Working times: [5]
      Worker L1_W1 (Capabilities: ["Painter"])
        Working times: [5]

Project: P2
  Stage: 1
    Location: L1
      Machine L1_M5 (Capabilities: ["Assembler"])
        Working times: [0, 1, 2, 3, 4]
      Worker L1_W4 (Capabilities: ["Assembler", "Inspector"])
        Working times: [0, 1, 2, 3, 4]
  Stage: 2
    Location: L1
      Machine L1_M1 (Capabilities: ["Painter"])
        Working times: [5, 6, 7, 8]
      Worker L1_W2 (Capabilities: ["Assembler"])
        Working times: [5, 6, 7, 8]
  Stage: 3
    Location: L1
      Machine L1_M5 (Capabilities: ["Assembler"])
        Working times: [9]
      Worker L1_W1 (Capabilities: ["Painter"])
    

In [ ]:
#Preparing the rest of the Parameters
using JuMP, HiGHS

# Create model
model = Model(HiGHS.Optimizer)

lambda_array = 

# Decision variables - now with location-specific workers and machines
@variable(model, y[p in projects, s in project_stages[p], l in locations, m in location_machines[l], t in T], Bin)  # Machine assignments
@variable(model, z[p in projects, s in project_stages[p], l in locations, wr in location_workers[l], t in T], Bin)  # Worker assignments
@variable(model, x[projects], Bin)  # Project acceptance
@variable(model, 0 <= stage_start[p in projects, s in project_stages[p]] <= get_project_deadline(p), Int)
@variable(model, I1[p in projects], Bin)
@variable(model, I2[p in projects, s in project_stages[p]], Bin)
@variable(model, I3[p in projects, s in project_stages[p]], Bin)

# Objective function
@objective(model, Max, sum( (get_project_revenue(p) + lambda * get_project_priotiy(p)) * x[p] for p in projects))

#Worker and Machine Uniqueness Constraints
@constraint(model, [l in locations, m in location_machines[l], t in T], sum(y[p,s,l,m,t] for p in projects, s in project_stages[p]) <= 1)
@constraint(model, [l in locations, wr in location_workers[l], t in T], sum(z[p,s,l,wr,t] for p in projects, s in project_stages[p]) <= 1)

#Machine and Worker Capability Constraints
for l in locations, p in projects, s in project_stages[p]

    stage_machines_needed = get_project_stage_machines(p,s)
    for m in location_machines[l]
        machine_capabilities = get_machine_capabilities(l,m)
        if isempty(intersect(stage_machines_needed, machine_capabilities)) ####Ensures if the machine has a list of skills, there still exists no identical skill in the stage
            @constraint(model, [t in T], y[p,s,l,m,t] == 0)
        end
    end

    stage_workers_needed = get_project_stage_workers(p, s)
    for wr in location_workers[l]
        worker_capabilities = get_worker_capabilities(l,wr)
        if isempty(intersect(worker_capabilities, stage_workers_needed))
            @constraint(model, [t in T], z[p,s,l,wr,t] == 0)
        end
    end
end

for p in projects
    for s in project_stages[p]
        if s > 1
            @constraint(model, stage_start[p, s] >= stage_start[p, s-1] + get_project_stage_duration(p, s-1))
        end
    end
end

#@constraint(model, [p in projects, s in project_stages[p] if s > 1, t in T], stage_start[p,s] >= stage_start[p,s-1] + get_project_stage_duration(p,s-1))

# =====> D1[p] = 1
#@constraint(model, [p in projects, s in last(project_stages[p]), t in T],  stage_start[p,s] <= get_project_deadline(p) - get_project_stage_duration(p,s))
@constraint(model, [p in projects, s in last(project_stages[p]), t in T],  stage_start[p,s] - (get_project_deadline(p) - get_project_stage_duration(p,s)) <= (get_project_deadline(p) - get_project_stage_duration(p,s))*(1-I1[p]))
#@constraint(model, [p in projects, s in last(project_stages[p]), t in T],  stage_start[p,s] - (get_project_deadline(p) - get_project_stage_duration(p,s)) >= -1*(get_project_deadline(p) - get_project_stage_duration(p,s))*I1[p] + .0001*(1-I1[p]))

# =====> D2[p] = 1
#@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) - get_project_stage_duration(p,s) >= 0)
@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) - get_project_stage_duration(p,s) >= (-get_project_stage_duration(p,s))*(1-I2[p,s]))
@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) <= get_project_stage_duration(p,s))
#@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(z[p,s,l,wr,t] for wr in location_workers[l], t in T) - get_project_stage_duration(p,s) <= (deadline*length(location_workers[l]) - get_project_stage_duration(p,s))*I2[p,s] - 0.0001*(1-I2[p,s]))

# =====> D3[p] = 1
#@constraint(model, [l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) >= get_project_stage_duration(p,s))
@constraint(model, [p in projects,l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) - get_project_stage_duration(p,s) >= -1*get_project_stage_duration(p,s)*(1-I3[p,s]))
@constraint(model, [p in projects,l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) <= get_project_stage_duration(p,s))
#@constraint(model, [p in projects, l in locations, s in project_stages[p]], sum(y[p,s,l,m,t] for t in T, m in location_machines[l]) - get_project_stage_duration(p,s) <= ((deadline+1)*length(location_machines[l]) - get_project_stage_duration(p,s))*I3[p,s] - 0.0001*(1-I3[p,s]))

#@constraint(model, [p in projects], I1[p] + sum(I2[p,s] for s in project_stages[p]) + sum(I3[p,s] for s in project_stages[p]) <= 2 * length(project_stages[p]) + x[p])

@constraint(model, [p in projects], I1[p] >= x[p])
@constraint(model, [p in projects, s in project_stages[p]], I2[p,s] >= x[p])
@constraint(model, [p in projects, s in project_stages[p]], I3[p,s] >= x[p])

# Add binary variables to represent if a time period is within a stage's duration
@variable(model, is_stage_active[p in projects, s in project_stages[p], t in T], Bin)

# Constraints to set is_stage_active based on stage_start and duration
for p in projects, s in project_stages[p]
    stage_duration = get_project_stage_duration(p,s)
    
    # Set is_stage_active to 1 only during the stage's duration
    for t in T
        # If t < stage_start[p,s], then is_stage_active must be 0
        @constraint(model, is_stage_active[p,s,t] <= 1 - (stage_start[p,s] - t)/deadline)
        
        # If t >= stage_start[p,s] + stage_duration, then is_stage_active must be 0
        @constraint(model, is_stage_active[p,s,t] <= 1 - (t - (stage_start[p,s] + stage_duration - 1))/deadline)
        
        # If stage_start[p,s] = 1, then t < stage_start[p,s] + stage_duration, then is_stage_active can be 1
        #@constraint(model, is_stage_active[p,s,t] >= (t - stage_start[p,s] + 1)/deadline)
        #@constraint(model, is_stage_active[p,s,t] >= (stage_start[p,s] + stage_duration - t)/deadline - 1)
    end

    # Link machine assignments to active time periods
    for l in locations, m in location_machines[l]
        @constraint(model, [t in T], y[p,s,l,m,t] <= is_stage_active[p,s,t])
    end
    
    # Link worker assignments to active time periods
    for l in locations, wr in location_workers[l]
        @constraint(model, [t in T], z[p,s,l,wr,t] <= is_stage_active[p,s,t])
    end
end

@variable(model, machine_assigned[p in projects, s in project_stages[p], l in locations, m in location_machines[l]], Bin)

# Link machine_assigned with y variables
@constraint(model, [p in projects, s in project_stages[p], l in locations, m in location_machines[l]], sum(y[p,s,l,m,t] for t in T) <= get_project_stage_duration(p,s) * machine_assigned[p,s,l,m])

# Ensure only one machine is assigned per stage
@constraint(model, [p in projects, s in project_stages[p], l in locations], sum(machine_assigned[p,s,l,m] for m in location_machines[l]) <= 1)

@variable(model, worker_assigned[p in projects, s in project_stages[p], l in locations, wr in location_workers[l]], Bin)

# Link machine_assigned with y variables
@constraint(model, [p in projects, s in project_stages[p], l in locations, wr in location_workers[l]], sum(z[p,s,l,wr,t] for t in T) <= get_project_stage_duration(p,s) * worker_assigned[p,s,l,wr])

# Ensure only one machine is assigned per stage
@constraint(model, [p in projects, s in project_stages[p], l in locations], sum(worker_assigned[p,s,l,wr] for wr in location_workers[l]) <= 1)

@constraint(model, [l in locations, r in get_all_resources_ids(l)], sum(get_project_resources_amount(p,r) * x[p] for p in projects) <= get_resource_amounts(l,r))

optimize!(model)
for p in projects
    println("Project: ", p, " ", value(x[p]))
end